In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import sequence

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
vocab_size = 10000
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=vocab_size)

17464789/17464789 [==============================] - 0s 0us/step


In [4]:
word_index = tf.keras.datasets.imdb.get_word_index()

1641221/1641221 [==============================] - 0s 0us/step


In [5]:
word_index = {k:(v+3) for k, v in word_index.items()}

word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNKNOWN>"] = 2
word_index["<UNUSED>"] = 3


reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [6]:
# 這邊可以調
review_length = 500

x_train = sequence.pad_sequences(x_train, maxlen = review_length)
x_test = sequence.pad_sequences(x_test, maxlen = review_length)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(25000, 500)
(25000,)
(25000, 500)
(25000,)


In [7]:
from tensorflow.keras import layers, models


In [8]:
rnn = models.Sequential(
    [
        layers.Embedding(
            input_dim = vocab_size,
            output_dim = 32,
            input_length = review_length,
        ),

        layers.SimpleRNN(
            units = 32,
        ),

        layers.Dense(
            units = 1,
            activation = "sigmoid"
        ),
    ],
    name = "RNN"
)

rnn.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.binary_crossentropy,
    metrics = ["accuracy"],
)

rnn.summary()

Model: "RNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 32)           320000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                2080      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 322113 (1.23 MB)
Trainable params: 322113 (1.23 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
history = rnn.fit(
    x_train,
    y_train,
    batch_size = 250,
    epochs = 10,
    validation_split = 0.2,
)

Epoch 1/10
80/80 [==============================] - 20s 231ms/step - loss: 0.6803 - accuracy: 0.5736 - val_loss: 0.6141 - val_accuracy: 0.7310
Epoch 2/10
80/80 [==============================] - 19s 241ms/step - loss: 0.4942 - accuracy: 0.7803 - val_loss: 0.4603 - val_accuracy: 0.7946
Epoch 3/10
80/80 [==============================] - 19s 236ms/step - loss: 0.3439 - accuracy: 0.8622 - val_loss: 0.3760 - val_accuracy: 0.8406
Epoch 4/10
80/80 [==============================] - 19s 235ms/step - loss: 0.2345 - accuracy: 0.9136 - val_loss: 0.3592 - val_accuracy: 0.8544
Epoch 5/10
80/80 [==============================] - 19s 235ms/step - loss: 0.1609 - accuracy: 0.9478 - val_loss: 0.3760 - val_accuracy: 0.8598
Epoch 6/10
80/80 [==============================] - 18s 225ms/step - loss: 0.1251 - accuracy: 0.9603 - val_loss: 0.3710 - val_accuracy: 0.8592
Epoch 7/10
80/80 [==============================] - 18s 223ms/step - loss: 0.0712 - accuracy: 0.9821 - val_loss: 0.4088 - val_accuracy: 0.8466

In [10]:
import json

history_dict = history.history

In [11]:
json.dump(history_dict, open("/content/drive/MyDrive/Colab Notebooks/imdb/history/rnn.json", 'w'))

In [12]:
rnn.save("/content/drive/MyDrive/Colab Notebooks/imdb/model/rnn.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
